In [3]:
import pandas as pd
from sqlalchemy import create_engine
import sqlite3
import psycopg2
from psycopg2 import sql
import json 

In [34]:
""" FUNCIONES """



# 1. Conexión con BD
def conexionBDPostgresSQL():
    # Se omiten los datos de conexión.
    conexion = psycopg2.connect(
        dbname="postgres",
        user="postgres",
        password="serRIB99",
        host="localhost",
        port="5432"
    )
    return conexion

# 1.2 Con SQLALCHEMY(dataframeGeneracion):
def conexionBD_SQLALCHEMY():
  # Datos de conexión a POSTGRESQL
  usuario = 'postgres'
  password = 'serRIB99'
  servidor = 'localhost'
  puerto = '5432'
  basedatos = 'postgres'
    
  # Crear la URL de conexión
  urlConexion = f'postgresql+psycopg2://{usuario}:{password}@{servidor}:{puerto}/{basedatos}'
  # Motor conexión Postgresql
  engine = create_engine(urlConexion)
  return engine

# 3. Función para ejecutar cualquier comando SQL
def ejecutarComandoSQL(comandoSQL):
  connection = conexionBDPostgresSQL()
  if connection is None:
      return
  try:
      cursor = connection.cursor()
      cursor.execute(comandoSQL)
      connection.commit()
      print(f"Comando ejecutado con éxito: {comandoSQL[0:50]}")
  except Exception as error:
      print(f"Error al ejecutar el comando: {error}")
  finally:
      if connection:
          cursor.close()
          connection.close()

# 2. Función para ejecutar cualquier comando SQL (SELECT)
def ejecutarComandoSQLSelect(comandoSQL):
  connection = conexionBDPostgresSQL()
  resultados = []
  if connection is None:
      return
  try:
      cursor = connection.cursor()
      cursor.execute(comandoSQL)
      # Obtener todos los resultados de la consulta
      resultados = cursor.fetchall()

      # Imprimir los resultados
      """for fila in resultados:
          print(fila)"""
      connection.commit()
      print(f"Comando ejecutado con éxito: {comandoSQL[0:50]}")
  except Exception as error:
      print(f"Error al ejecutar el comando: \n{error}")
  finally:
    if connection:
        cursor.close()
        connection.close()
    return resultados



""" Funciones - Acondicionamiento de datos Climatología """
# 1. Inserción en BBDD
def Preprocesado_Climatologia_InsertBD(dataframeClimatologia):
  motorConexion = conexionBD_SQLALCHEMY()

  # Tabla y Esquema de BD
  esquemaBD = 'tfm'
  tabla = 'climatologia_procesada'

  # Insertar en BBDD desde un Dataframe
  dataframeClimatologia.to_sql(tabla, motorConexion, schema=esquemaBD, if_exists='append', index=False)

# 2. Función que obtiene los datos en formato original y los procesa para acondicionarlos al formato final.
def Preprocesamiento_Climatologia_ToDataframe():
    # Comando SQL para descargar datos de la BBDD
    comandoSQL = """SELECT c.fecha, c.precipitaciones, c.sol, c.temperaturamedia,
        c.temperaturamaxima, c.temperaturaminima, c.vientoracha, c.vientovelmedia,
        c.humedadrelativamedia, c.humedadrelativamaxima, c.humedadrelativaminima,
        c.presionmaxima, c.presionminima
        FROM tfm.climatologiaprovincias c
        WHERE c.fecha between '2011-01-01' AND '2025-02-01'
        ORDER BY c.fecha ASC
    """
    registrosClimaProvincias = ejecutarComandoSQLSelect(comandoSQL)

    # Crear Dataframe a partir de los datos
    dataframeClima = pd.DataFrame(registrosClimaProvincias, columns=['fecha', 'precipitacionesmedia', 'horassolmedia',
        'temperaturamedia', 'temperaturamaxmedia', 'temperaturaminmedia',
        'rachamedia', 'vientovelmedia',
        'humedadrelmedia', 'humedadrelmaxmedia',
        'humedadrelminmedia', 'presionmaxmedia', 'presionminmedia'])

    # Fecha: tipo date
    dataframeClima['fecha'] = pd.to_datetime(dataframeClima['fecha'])

    # Agrupar los datos, por fecha, y aplicar una función agregada
    dataframeClima = dataframeClima.groupby('fecha').agg({
        'temperaturamedia': 'mean', # Media
        'temperaturamaxmedia': 'mean',
        'temperaturaminmedia': 'mean',
        'precipitacionesmedia': 'mean', 
        'horassolmedia': 'mean',
        'rachamedia': 'mean',
        'vientovelmedia': 'mean',
        'humedadrelmedia': 'mean',
        'humedadrelmaxmedia': 'mean',
        'humedadrelminmedia': 'mean',
        'presionmaxmedia': 'mean',
        'presionminmedia': 'mean'
    }).reset_index()


    # DataFrame a CSV
    dataframeClima.to_csv('Climatologia_procesada.csv', index=False)

    # Insertar en BD
    Preprocesado_Climatologia_InsertBD(dataframeClima)
    


In [36]:
""" MAIN """
ejecutarComandoSQL("TRUNCATE TABLE tfm.climatologia_procesada;")

resultados_climatologicos = Preprocesamiento_Climatologia_ToDataframe()

ejecutarComandoSQLSelect("SELECT COUNT(*) FROM tfm.climatologia_procesada")

Comando ejecutado con éxito: TRUNCATE TABLE tfm.climatologia_procesada;
Comando ejecutado con éxito: SELECT c.fecha, c.precipitaciones, c.sol, c.temper
Comando ejecutado con éxito: SELECT COUNT(*) FROM tfm.climatologia_procesada
Comando ejecutado con éxito: SELECT * FROM tfm.climatologia_procesada


[(datetime.datetime(2011, 1, 1, 0, 0),
  Decimal('0.5432801822323462'),
  Decimal('2.2035714285714287'),
  Decimal('9.304666666666668'),
  Decimal('12.577161862527717'),
  Decimal('6.0440000000000005'),
  Decimal('5.622408963585435'),
  Decimal('1.5112328767123289'),
  Decimal('86.44063926940639'),
  Decimal('96.56459330143541'),
  Decimal('73.04066985645933'),
  Decimal('964.0009174311928'),
  Decimal('961.3660550458715')),
 (datetime.datetime(2011, 1, 2, 0, 0),
  Decimal('0.3672645739910314'),
  Decimal('3.4535714285714287'),
  Decimal('8.518444444444444'),
  Decimal('12.329333333333333'),
  Decimal('4.714222222222222'),
  Decimal('7.271030640668524'),
  Decimal('2.0383783783783787'),
  Decimal('81.71004566210046'),
  Decimal('95.11483253588517'),
  Decimal('65.94976076555024'),
  Decimal('962.2917431192661'),
  Decimal('959.9073394495413')),
 (datetime.datetime(2011, 1, 3, 0, 0),
  Decimal('1.0675555555555556'),
  Decimal('4.641935483870968'),
  Decimal('6.320088300220751'),
  Decim